# MoA

Fork from [here](https://www.kaggle.com/simakov/keras-multilabel-neural-network-v1-2)

## ライブラリ

In [1]:
import sys

sys.path.append("../input/iterative-stratification/iterative-stratification-master")
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import datetime
import gc
import os
import random
from time import time

import numpy as np
import optuna
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import log_loss
from sklearn.preprocessing import QuantileTransformer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

## 関数

In [3]:
def fix_seed(seed=2020):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


random_seed = 22
fix_seed(random_seed)

In [4]:
# [Fast Numpy Log Loss] https://www.kaggle.com/gogo827jz/optimise-blending-weights-4-5x-faster-log-loss
def metric(y_true, y_pred):
    loss = 0
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    for i in range(y_pred.shape[1]):
        loss += -np.mean(y_true[:, i] * np.log(y_pred_clip[:, i]) + (1 - y_true[:, i]) * np.log(1 - y_pred_clip[:, i]))
    return loss / y_pred.shape[1]

## データロード

In [5]:
# train_df = pd.read_csv("../input/lish-moa/train_features.csv")
# test_df = pd.read_csv("../input/lish-moa/test_features.csv")
# target_df = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
# non_target_df = pd.read_csv("../input/lish-moa/train_targets_nonscored.csv")
# submit_df = pd.read_csv("../input/lish-moa/sample_submission.csv")

In [6]:
# train = train_df.copy()
# test = test_df.copy()

## 前処理

In [7]:
# train.loc[:, "cp_dose"] = train.loc[:, "cp_dose"].map({"D1": 0, "D2": 1})
# test.loc[:, "cp_dose"] = test.loc[:, "cp_dose"].map({"D1": 0, "D2": 1})

### cp_type が ctrl_vehicle なものは MoA を持たない

ので、学習から除外する

In [8]:
# target_df = target_df.loc[train["cp_type"] != "ctl_vehicle"].reset_index(drop=True)
# train = train.loc[train["cp_type"] != "ctl_vehicle"].reset_index(drop=True)

In [9]:
# train = train.drop("cp_type", axis=1)
# test = test.drop("cp_type", axis=1)

In [10]:
# del train["sig_id"]
# del target_df["sig_id"]
# del test["sig_id"]

### Rank Gauss

https://www.kaggle.com/nayuts/moa-pytorch-nn-pca-rankgauss

連続値を特定の範囲の閉域に押し込めて、分布の偏りを解消する方法です。

In [11]:
# g_cols = [col for col in train_df.columns if col.startswith("g-")]
# c_cols = [col for col in train_df.columns if col.startswith("c-")]
#
# for col in g_cols + c_cols:
#    transformer = QuantileTransformer(n_quantiles=100, random_state=random_seed, output_distribution="normal")
#
#    vec_len = len(train[col].values)
#    vec_len_test = len(test[col].values)
#
#    raw_vec = train[col].values.reshape(vec_len, 1)
#    transformer.fit(raw_vec)
#
#    train[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
#    test[col] = transformer.transform(test[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

### PCA features (+ Existing features)

In [12]:
## g-
# n_comp = 600
#
# data = pd.concat([pd.DataFrame(train[g_cols]), pd.DataFrame(test[g_cols])])
# data2 = PCA(n_components=n_comp, random_state=random_seed).fit_transform(data[g_cols])
# train2 = data2[: train.shape[0]]
# test2 = data2[-test.shape[0] :]
#
# train2 = pd.DataFrame(train2, columns=[f"pca_G-{i}" for i in range(n_comp)])
# test2 = pd.DataFrame(test2, columns=[f"pca_G-{i}" for i in range(n_comp)])
#
# train.drop(g_cols, axis=1, inplace=True)
# test.drop(g_cols, axis=1, inplace=True)
#
# train = pd.concat((train, train2), axis=1)
# test = pd.concat((test, test2), axis=1)

In [13]:
## c-
# n_comp = 50
#
# data = pd.concat([pd.DataFrame(train[c_cols]), pd.DataFrame(test[c_cols])])
# data2 = PCA(n_components=n_comp, random_state=random_seed).fit_transform(data[c_cols])
# train2 = data2[: train.shape[0]]
# test2 = data2[-test.shape[0] :]
#
# train2 = pd.DataFrame(train2, columns=[f"pca_C-{i}" for i in range(n_comp)])
# test2 = pd.DataFrame(test2, columns=[f"pca_C-{i}" for i in range(n_comp)])
#
# train.drop(c_cols, axis=1, inplace=True)
# test.drop(c_cols, axis=1, inplace=True)
#
# train = pd.concat((train, train2), axis=1)
# test = pd.concat((test, test2), axis=1)

### feature Selection using Variance Encoding

分散がしきい値以下の特徴量を捨てます。

In [14]:
# var_thresh = VarianceThreshold(threshold=0.5)
#
# data = train.append(test)
# data_transformed = var_thresh.fit_transform(data.iloc[:, 2:])
#
# train_transformed = data_transformed[: train.shape[0]]
# test_transformed = data_transformed[-test.shape[0] :]
#
#
# train = pd.DataFrame(train[["cp_time", "cp_dose"]].values.reshape(-1, 2), columns=["cp_time", "cp_dose"])
# train = pd.concat([train, pd.DataFrame(train_transformed)], axis=1)
#
#
# test = pd.DataFrame(test[["cp_time", "cp_dose"]].values.reshape(-1, 2), columns=["cp_time", "cp_dose"])
# test = pd.concat([test, pd.DataFrame(test_transformed)], axis=1)

## モデル作成

In [15]:
def create_model():
    model = tf.keras.Sequential(
        [
            L.Input(len(train.columns)),
            L.BatchNormalization(),
            tfa.layers.WeightNormalization(L.Dense(2048, activation="relu")),
            L.BatchNormalization(),
            L.Dropout(0.4),
            tfa.layers.WeightNormalization(L.Dense(2048, activation="relu")),
            L.BatchNormalization(),
            L.Dropout(0.4),
            tfa.layers.WeightNormalization(L.Dense(206, activation="sigmoid")),
        ]
    )
    model.compile(
        optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(), sync_period=10),
        loss="binary_crossentropy",
    )
    return model

### TabNet for Tensorflow for MultiLabel

https://www.kaggle.com/gogo827jz/moa-stacked-tabnet-baseline-tensorflow-2-0#Model-Functions

In [16]:
def register_keras_custom_object(cls):
    tf.keras.utils.get_custom_objects()[cls.__name__] = cls
    return cls

In [17]:
def glu(x, n_units=None):
    """Generalized linear unit nonlinear activation."""
    if n_units is None:
        n_units = tf.shape(x)[-1] // 2

    return x[..., :n_units] * tf.nn.sigmoid(x[..., n_units:])

In [18]:
"""
Code replicated from https://github.com/tensorflow/addons/blob/master/tensorflow_addons/activations/sparsemax.py
"""


@register_keras_custom_object
@tf.function
def sparsemax(logits, axis):
    """Sparsemax activation function [1].
    For each batch `i` and class `j` we have
      $$sparsemax[i, j] = max(logits[i, j] - tau(logits[i, :]), 0)$$
    [1]: https://arxiv.org/abs/1602.02068
    Args:
        logits: Input tensor.
        axis: Integer, axis along which the sparsemax operation is applied.
    Returns:
        Tensor, output of sparsemax transformation. Has the same type and
        shape as `logits`.
    Raises:
        ValueError: In case `dim(logits) == 1`.
    """
    logits = tf.convert_to_tensor(logits, name="logits")

    # We need its original shape for shape inference.
    shape = logits.get_shape()
    rank = shape.rank
    is_last_axis = (axis == -1) or (axis == rank - 1)

    if is_last_axis:
        output = _compute_2d_sparsemax(logits)
        output.set_shape(shape)
        return output

    # If dim is not the last dimension, we have to do a transpose so that we can
    # still perform softmax on its last dimension.

    # Swap logits' dimension of dim and its last dimension.
    rank_op = tf.rank(logits)
    axis_norm = axis % rank
    logits = _swap_axis(logits, axis_norm, tf.math.subtract(rank_op, 1))

    # Do the actual softmax on its last dimension.
    output = _compute_2d_sparsemax(logits)
    output = _swap_axis(output, axis_norm, tf.math.subtract(rank_op, 1))

    # Make shape inference work since transpose may erase its static shape.
    output.set_shape(shape)
    return output

In [19]:
def _swap_axis(logits, dim_index, last_index, **kwargs):
    return tf.transpose(
        logits,
        tf.concat(
            [
                tf.range(dim_index),
                [last_index],
                tf.range(dim_index + 1, last_index),
                [dim_index],
            ],
            0,
        ),
        **kwargs,
    )

In [20]:
def _compute_2d_sparsemax(logits):
    """Performs the sparsemax operation when axis=-1."""
    shape_op = tf.shape(logits)
    obs = tf.math.reduce_prod(shape_op[:-1])
    dims = shape_op[-1]

    # In the paper, they call the logits z.
    # The mean(logits) can be substracted from logits to make the algorithm
    # more numerically stable. the instability in this algorithm comes mostly
    # from the z_cumsum. Substacting the mean will cause z_cumsum to be close
    # to zero. However, in practise the numerical instability issues are very
    # minor and substacting the mean causes extra issues with inf and nan
    # input.
    # Reshape to [obs, dims] as it is almost free and means the remanining
    # code doesn't need to worry about the rank.
    z = tf.reshape(logits, [obs, dims])

    # sort z
    z_sorted, _ = tf.nn.top_k(z, k=dims)

    # calculate k(z)
    z_cumsum = tf.math.cumsum(z_sorted, axis=-1)
    k = tf.range(1, tf.cast(dims, logits.dtype) + 1, dtype=logits.dtype)
    z_check = 1 + k * z_sorted > z_cumsum
    # because the z_check vector is always [1,1,...1,0,0,...0] finding the
    # (index + 1) of the last `1` is the same as just summing the number of 1.
    k_z = tf.math.reduce_sum(tf.cast(z_check, tf.int32), axis=-1)

    # calculate tau(z)
    # If there are inf values or all values are -inf, the k_z will be zero,
    # this is mathematically invalid and will also cause the gather_nd to fail.
    # Prevent this issue for now by setting k_z = 1 if k_z = 0, this is then
    # fixed later (see p_safe) by returning p = nan. This results in the same
    # behavior as softmax.
    k_z_safe = tf.math.maximum(k_z, 1)
    indices = tf.stack([tf.range(0, obs), tf.reshape(k_z_safe, [-1]) - 1], axis=1)
    tau_sum = tf.gather_nd(z_cumsum, indices)
    tau_z = (tau_sum - 1) / tf.cast(k_z, logits.dtype)

    # calculate p
    p = tf.math.maximum(tf.cast(0, logits.dtype), z - tf.expand_dims(tau_z, -1))
    # If k_z = 0 or if z = nan, then the input is invalid
    p_safe = tf.where(
        tf.expand_dims(
            tf.math.logical_or(tf.math.equal(k_z, 0), tf.math.is_nan(z_cumsum[:, -1])),
            axis=-1,
        ),
        tf.fill([obs, dims], tf.cast(float("nan"), logits.dtype)),
        p,
    )

    # Reshape back to original size
    p_safe = tf.reshape(p_safe, shape_op)
    return p_safe

In [21]:
"""
Code replicated from https://github.com/tensorflow/addons/blob/master/tensorflow_addons/layers/normalizations.py
"""


@register_keras_custom_object
class GroupNormalization(tf.keras.layers.Layer):
    """Group normalization layer.
    Group Normalization divides the channels into groups and computes
    within each group the mean and variance for normalization.
    Empirically, its accuracy is more stable than batch norm in a wide
    range of small batch sizes, if learning rate is adjusted linearly
    with batch sizes.
    Relation to Layer Normalization:
    If the number of groups is set to 1, then this operation becomes identical
    to Layer Normalization.
    Relation to Instance Normalization:
    If the number of groups is set to the
    input dimension (number of groups is equal
    to number of channels), then this operation becomes
    identical to Instance Normalization.
    Arguments
        groups: Integer, the number of groups for Group Normalization.
            Can be in the range [1, N] where N is the input dimension.
            The input dimension must be divisible by the number of groups.
        axis: Integer, the axis that should be normalized.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    Output shape
        Same shape as input.
    References
        - [Group Normalization](https://arxiv.org/abs/1803.08494)
    """

    def __init__(
        self,
        groups: int = 2,
        axis: int = -1,
        epsilon: float = 1e-3,
        center: bool = True,
        scale: bool = True,
        beta_initializer="zeros",
        gamma_initializer="ones",
        beta_regularizer=None,
        gamma_regularizer=None,
        beta_constraint=None,
        gamma_constraint=None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.groups = groups
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = tf.keras.initializers.get(beta_initializer)
        self.gamma_initializer = tf.keras.initializers.get(gamma_initializer)
        self.beta_regularizer = tf.keras.regularizers.get(beta_regularizer)
        self.gamma_regularizer = tf.keras.regularizers.get(gamma_regularizer)
        self.beta_constraint = tf.keras.constraints.get(beta_constraint)
        self.gamma_constraint = tf.keras.constraints.get(gamma_constraint)
        self._check_axis()

    def build(self, input_shape):

        self._check_if_input_shape_is_none(input_shape)
        self._set_number_of_groups_for_instance_norm(input_shape)
        self._check_size_of_dimensions(input_shape)
        self._create_input_spec(input_shape)

        self._add_gamma_weight(input_shape)
        self._add_beta_weight(input_shape)
        self.built = True
        super().build(input_shape)

    def call(self, inputs, training=None):
        # Training=none is just for compat with batchnorm signature call
        input_shape = tf.keras.backend.int_shape(inputs)
        tensor_input_shape = tf.shape(inputs)

        reshaped_inputs, group_shape = self._reshape_into_groups(inputs, input_shape, tensor_input_shape)

        normalized_inputs = self._apply_normalization(reshaped_inputs, input_shape)

        outputs = tf.reshape(normalized_inputs, tensor_input_shape)

        return outputs

    def get_config(self):
        config = {
            "groups": self.groups,
            "axis": self.axis,
            "epsilon": self.epsilon,
            "center": self.center,
            "scale": self.scale,
            "beta_initializer": tf.keras.initializers.serialize(self.beta_initializer),
            "gamma_initializer": tf.keras.initializers.serialize(self.gamma_initializer),
            "beta_regularizer": tf.keras.regularizers.serialize(self.beta_regularizer),
            "gamma_regularizer": tf.keras.regularizers.serialize(self.gamma_regularizer),
            "beta_constraint": tf.keras.constraints.serialize(self.beta_constraint),
            "gamma_constraint": tf.keras.constraints.serialize(self.gamma_constraint),
        }
        base_config = super().get_config()
        return {**base_config, **config}

    def compute_output_shape(self, input_shape):
        return input_shape

    def _reshape_into_groups(self, inputs, input_shape, tensor_input_shape):

        group_shape = [tensor_input_shape[i] for i in range(len(input_shape))]
        group_shape[self.axis] = input_shape[self.axis] // self.groups
        group_shape.insert(self.axis, self.groups)
        group_shape = tf.stack(group_shape)
        reshaped_inputs = tf.reshape(inputs, group_shape)
        return reshaped_inputs, group_shape

    def _apply_normalization(self, reshaped_inputs, input_shape):

        group_shape = tf.keras.backend.int_shape(reshaped_inputs)
        group_reduction_axes = list(range(1, len(group_shape)))
        axis = -2 if self.axis == -1 else self.axis - 1
        group_reduction_axes.pop(axis)

        mean, variance = tf.nn.moments(reshaped_inputs, group_reduction_axes, keepdims=True)

        gamma, beta = self._get_reshaped_weights(input_shape)
        normalized_inputs = tf.nn.batch_normalization(
            reshaped_inputs,
            mean=mean,
            variance=variance,
            scale=gamma,
            offset=beta,
            variance_epsilon=self.epsilon,
        )
        return normalized_inputs

    def _get_reshaped_weights(self, input_shape):
        broadcast_shape = self._create_broadcast_shape(input_shape)
        gamma = None
        beta = None
        if self.scale:
            gamma = tf.reshape(self.gamma, broadcast_shape)

        if self.center:
            beta = tf.reshape(self.beta, broadcast_shape)
        return gamma, beta

    def _check_if_input_shape_is_none(self, input_shape):
        dim = input_shape[self.axis]
        if dim is None:
            raise ValueError(
                "Axis " + str(self.axis) + " of "
                "input tensor should have a defined dimension "
                "but the layer received an input with shape " + str(input_shape) + "."
            )

    def _set_number_of_groups_for_instance_norm(self, input_shape):
        dim = input_shape[self.axis]

        if self.groups == -1:
            self.groups = dim

    def _check_size_of_dimensions(self, input_shape):

        dim = input_shape[self.axis]
        if dim < self.groups:
            raise ValueError(
                "Number of groups (" + str(self.groups) + ") cannot be "
                "more than the number of channels (" + str(dim) + ")."
            )

        if dim % self.groups != 0:
            raise ValueError(
                "Number of groups (" + str(self.groups) + ") must be a "
                "multiple of the number of channels (" + str(dim) + ")."
            )

    def _check_axis(self):

        if self.axis == 0:
            raise ValueError(
                "You are trying to normalize your batch axis. Do you want to "
                "use tf.layer.batch_normalization instead"
            )

    def _create_input_spec(self, input_shape):

        dim = input_shape[self.axis]
        self.input_spec = tf.keras.layers.InputSpec(ndim=len(input_shape), axes={self.axis: dim})

    def _add_gamma_weight(self, input_shape):

        dim = input_shape[self.axis]
        shape = (dim,)

        if self.scale:
            self.gamma = self.add_weight(
                shape=shape,
                name="gamma",
                initializer=self.gamma_initializer,
                regularizer=self.gamma_regularizer,
                constraint=self.gamma_constraint,
            )
        else:
            self.gamma = None

    def _add_beta_weight(self, input_shape):

        dim = input_shape[self.axis]
        shape = (dim,)

        if self.center:
            self.beta = self.add_weight(
                shape=shape,
                name="beta",
                initializer=self.beta_initializer,
                regularizer=self.beta_regularizer,
                constraint=self.beta_constraint,
            )
        else:
            self.beta = None

    def _create_broadcast_shape(self, input_shape):
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis] // self.groups
        broadcast_shape.insert(self.axis, self.groups)
        return broadcast_shape

In [22]:
class TransformBlock(tf.keras.Model):
    def __init__(self, features, norm_type, momentum=0.9, virtual_batch_size=None, groups=2, block_name="", **kwargs):
        super(TransformBlock, self).__init__(**kwargs)

        self.features = features
        self.norm_type = norm_type
        self.momentum = momentum
        self.groups = groups
        self.virtual_batch_size = virtual_batch_size

        self.transform = tf.keras.layers.Dense(self.features, use_bias=False, name=f"transformblock_dense_{block_name}")

        if norm_type == "batch":
            self.bn = tf.keras.layers.BatchNormalization(
                axis=-1,
                momentum=momentum,
                virtual_batch_size=virtual_batch_size,
                name=f"transformblock_bn_{block_name}",
            )

        else:
            self.bn = GroupNormalization(axis=-1, groups=self.groups, name=f"transformblock_gn_{block_name}")

    def call(self, inputs, training=None):
        x = self.transform(inputs)
        x = self.bn(x, training=training)
        return x

In [23]:
class TabNet(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        feature_dim=64,
        output_dim=64,
        num_features=None,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=2,
        epsilon=1e-5,
        **kwargs,
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'batch' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(TabNet, self).__init__(**kwargs)

        # Input checks
        if feature_columns is not None:
            if type(feature_columns) not in (list, tuple):
                raise ValueError("`feature_columns` must be a list or a tuple.")

            if len(feature_columns) == 0:
                raise ValueError("`feature_columns` must be contain at least 1 tf.feature_column !")

            if num_features is None:
                num_features = len(feature_columns)
            else:
                num_features = int(num_features)

        else:
            if num_features is None:
                raise ValueError("If `feature_columns` is None, then `num_features` cannot be None.")

        if num_decision_steps < 1:
            raise ValueError("Num decision steps must be greater than 0.")

        if feature_dim < output_dim:
            raise ValueError("To compute `features_for_coef`, feature_dim must be larger than output dim")

        feature_dim = int(feature_dim)
        output_dim = int(output_dim)
        num_decision_steps = int(num_decision_steps)
        relaxation_factor = float(relaxation_factor)
        sparsity_coefficient = float(sparsity_coefficient)
        batch_momentum = float(batch_momentum)
        num_groups = max(1, int(num_groups))
        epsilon = float(epsilon)

        if relaxation_factor < 0.0:
            raise ValueError("`relaxation_factor` cannot be negative !")

        if sparsity_coefficient < 0.0:
            raise ValueError("`sparsity_coefficient` cannot be negative !")

        if virtual_batch_size is not None:
            virtual_batch_size = int(virtual_batch_size)

        if norm_type not in ["batch", "group"]:
            raise ValueError("`norm_type` must be either `batch` or `group`")

        self.feature_columns = feature_columns
        self.num_features = num_features
        self.feature_dim = feature_dim
        self.output_dim = output_dim

        self.num_decision_steps = num_decision_steps
        self.relaxation_factor = relaxation_factor
        self.sparsity_coefficient = sparsity_coefficient
        self.norm_type = norm_type
        self.batch_momentum = batch_momentum
        self.virtual_batch_size = virtual_batch_size
        self.num_groups = num_groups
        self.epsilon = epsilon

        # if num_decision_steps > 1:
        # features_for_coeff = feature_dim - output_dim
        # print(f"[TabNet]: {features_for_coeff} features will be used for decision steps.")

        if self.feature_columns is not None:
            self.input_features = tf.keras.layers.DenseFeatures(feature_columns, trainable=True)

            if self.norm_type == "batch":
                self.input_bn = tf.keras.layers.BatchNormalization(axis=-1, momentum=batch_momentum, name="input_bn")
            else:
                self.input_bn = GroupNormalization(axis=-1, groups=self.num_groups, name="input_gn")

        else:
            self.input_features = None
            self.input_bn = None

        self.transform_f1 = TransformBlock(
            2 * self.feature_dim,
            self.norm_type,
            self.batch_momentum,
            self.virtual_batch_size,
            self.num_groups,
            block_name="f1",
        )

        self.transform_f2 = TransformBlock(
            2 * self.feature_dim,
            self.norm_type,
            self.batch_momentum,
            self.virtual_batch_size,
            self.num_groups,
            block_name="f2",
        )

        self.transform_f3_list = [
            TransformBlock(
                2 * self.feature_dim,
                self.norm_type,
                self.batch_momentum,
                self.virtual_batch_size,
                self.num_groups,
                block_name=f"f3_{i}",
            )
            for i in range(self.num_decision_steps)
        ]

        self.transform_f4_list = [
            TransformBlock(
                2 * self.feature_dim,
                self.norm_type,
                self.batch_momentum,
                self.virtual_batch_size,
                self.num_groups,
                block_name=f"f4_{i}",
            )
            for i in range(self.num_decision_steps)
        ]

        self.transform_coef_list = [
            TransformBlock(
                self.num_features,
                self.norm_type,
                self.batch_momentum,
                self.virtual_batch_size,
                self.num_groups,
                block_name=f"coef_{i}",
            )
            for i in range(self.num_decision_steps - 1)
        ]

        self._step_feature_selection_masks = None
        self._step_aggregate_feature_selection_mask = None

    def call(self, inputs, training=None):
        if self.input_features is not None:
            features = self.input_features(inputs)
            features = self.input_bn(features, training=training)

        else:
            features = inputs

        batch_size = tf.shape(features)[0]
        self._step_feature_selection_masks = []
        self._step_aggregate_feature_selection_mask = None

        # Initializes decision-step dependent variables.
        output_aggregated = tf.zeros([batch_size, self.output_dim])
        masked_features = features
        mask_values = tf.zeros([batch_size, self.num_features])
        aggregated_mask_values = tf.zeros([batch_size, self.num_features])
        complementary_aggregated_mask_values = tf.ones([batch_size, self.num_features])

        total_entropy = 0.0
        entropy_loss = 0.0

        for ni in range(self.num_decision_steps):
            # Feature transformer with two shared and two decision step dependent
            # blocks is used below.=
            transform_f1 = self.transform_f1(masked_features, training=training)
            transform_f1 = glu(transform_f1, self.feature_dim)

            transform_f2 = self.transform_f2(transform_f1, training=training)
            transform_f2 = (glu(transform_f2, self.feature_dim) + transform_f1) * tf.math.sqrt(0.5)

            transform_f3 = self.transform_f3_list[ni](transform_f2, training=training)
            transform_f3 = (glu(transform_f3, self.feature_dim) + transform_f2) * tf.math.sqrt(0.5)

            transform_f4 = self.transform_f4_list[ni](transform_f3, training=training)
            transform_f4 = (glu(transform_f4, self.feature_dim) + transform_f3) * tf.math.sqrt(0.5)

            if ni > 0 or self.num_decision_steps == 1:
                decision_out = tf.nn.relu(transform_f4[:, : self.output_dim])

                # Decision aggregation.
                output_aggregated += decision_out

                # Aggregated masks are used for visualization of the
                # feature importance attributes.
                scale_agg = tf.reduce_sum(decision_out, axis=1, keepdims=True)

                if self.num_decision_steps > 1:
                    scale_agg = scale_agg / tf.cast(self.num_decision_steps - 1, tf.float32)

                aggregated_mask_values += mask_values * scale_agg

            features_for_coef = transform_f4[:, self.output_dim :]

            if ni < (self.num_decision_steps - 1):
                # Determines the feature masks via linear and nonlinear
                # transformations, taking into account of aggregated feature use.
                mask_values = self.transform_coef_list[ni](features_for_coef, training=training)
                mask_values *= complementary_aggregated_mask_values
                mask_values = sparsemax(mask_values, axis=-1)

                # Relaxation factor controls the amount of reuse of features between
                # different decision blocks and updated with the values of
                # coefficients.
                complementary_aggregated_mask_values *= self.relaxation_factor - mask_values

                # Entropy is used to penalize the amount of sparsity in feature
                # selection.
                total_entropy += tf.reduce_mean(
                    tf.reduce_sum(-mask_values * tf.math.log(mask_values + self.epsilon), axis=1)
                ) / (tf.cast(self.num_decision_steps - 1, tf.float32))

                # Add entropy loss
                entropy_loss = total_entropy

                # Feature selection.
                masked_features = tf.multiply(mask_values, features)

                # Visualization of the feature selection mask at decision step ni
                # tf.summary.image(
                #     "Mask for step" + str(ni),
                #     tf.expand_dims(tf.expand_dims(mask_values, 0), 3),
                #     max_outputs=1)
                mask_at_step_i = tf.expand_dims(tf.expand_dims(mask_values, 0), 3)
                self._step_feature_selection_masks.append(mask_at_step_i)

            else:
                # This branch is needed for correct compilation by tf.autograph
                entropy_loss = 0.0

        # Adds the loss automatically
        self.add_loss(self.sparsity_coefficient * entropy_loss)

        # Visualization of the aggregated feature importances
        # tf.summary.image(
        #     "Aggregated mask",
        #     tf.expand_dims(tf.expand_dims(aggregated_mask_values, 0), 3),
        #     max_outputs=1)

        agg_mask = tf.expand_dims(tf.expand_dims(aggregated_mask_values, 0), 3)
        self._step_aggregate_feature_selection_mask = agg_mask

        return output_aggregated

    @property
    def feature_selection_masks(self):
        return self._step_feature_selection_masks

    @property
    def aggregate_feature_selection_mask(self):
        return self._step_aggregate_feature_selection_mask

In [24]:
class TabNetClassifier(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_classes,
        num_features=None,
        feature_dim=64,
        output_dim=64,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=1,
        epsilon=1e-5,
        multi_label=False,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_classes: Number of classes.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'group' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(TabNetClassifier, self).__init__(**kwargs)

        self.num_classes = num_classes

        self.tabnet = TabNet(
            feature_columns=feature_columns,
            num_features=num_features,
            feature_dim=feature_dim,
            output_dim=output_dim,
            num_decision_steps=num_decision_steps,
            relaxation_factor=relaxation_factor,
            sparsity_coefficient=sparsity_coefficient,
            norm_type=norm_type,
            batch_momentum=batch_momentum,
            virtual_batch_size=virtual_batch_size,
            num_groups=num_groups,
            epsilon=epsilon,
            **kwargs
        )

        if multi_label:

            self.clf = tf.keras.layers.Dense(num_classes, activation="sigmoid", use_bias=False, name="classifier")

        else:

            self.clf = tf.keras.layers.Dense(num_classes, activation="softmax", use_bias=False, name="classifier")

    def call(self, inputs, training=None):
        self.activations = self.tabnet(inputs, training=training)
        out = self.clf(self.activations)

        return out

    def summary(self, *super_args, **super_kwargs):
        super().summary(*super_args, **super_kwargs)
        self.tabnet.summary(*super_args, **super_kwargs)

In [25]:
class TabNetRegressor(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_regressors,
        num_features=None,
        feature_dim=64,
        output_dim=64,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=1,
        epsilon=1e-5,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_regressors: Number of regression variables.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'group' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(TabNetRegressor, self).__init__(**kwargs)

        self.num_regressors = num_regressors

        self.tabnet = TabNet(
            feature_columns=feature_columns,
            num_features=num_features,
            feature_dim=feature_dim,
            output_dim=output_dim,
            num_decision_steps=num_decision_steps,
            relaxation_factor=relaxation_factor,
            sparsity_coefficient=sparsity_coefficient,
            norm_type=norm_type,
            batch_momentum=batch_momentum,
            virtual_batch_size=virtual_batch_size,
            num_groups=num_groups,
            epsilon=epsilon,
            **kwargs
        )

        self.regressor = tf.keras.layers.Dense(num_regressors, use_bias=False, name="regressor")

    def call(self, inputs, training=None):
        self.activations = self.tabnet(inputs, training=training)
        out = self.regressor(self.activations)
        return out

    def summary(self, *super_args, **super_kwargs):
        super().summary(*super_args, **super_kwargs)
        self.tabnet.summary(*super_args, **super_kwargs)

In [26]:
# Aliases
TabNetClassification = TabNetClassifier
TabNetRegression = TabNetRegressor

In [27]:
class StackedTabNet(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_layers=1,
        feature_dim=64,
        output_dim=64,
        num_features=None,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=2,
        epsilon=1e-5,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        Stacked variant of the TabNet model, which stacks multiple TabNets into a singular model.
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_layers: Number of TabNets to stack together.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer. Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
                Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'batch' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(StackedTabNet, self).__init__(**kwargs)

        if num_layers < 1:
            raise ValueError("`num_layers` cannot be less than 1")

        if type(feature_dim) not in [list, tuple]:
            feature_dim = [feature_dim] * num_layers

        if type(output_dim) not in [list, tuple]:
            output_dim = [output_dim] * num_layers

        if len(feature_dim) != num_layers:
            raise ValueError("`feature_dim` must be a list of length `num_layers`")

        if len(output_dim) != num_layers:
            raise ValueError("`output_dim` must be a list of length `num_layers`")

        self.num_layers = num_layers

        layers = []
        layers.append(
            TabNet(
                feature_columns=feature_columns,
                num_features=num_features,
                feature_dim=feature_dim[0],
                output_dim=output_dim[0],
                num_decision_steps=num_decision_steps,
                relaxation_factor=relaxation_factor,
                sparsity_coefficient=sparsity_coefficient,
                norm_type=norm_type,
                batch_momentum=batch_momentum,
                virtual_batch_size=virtual_batch_size,
                num_groups=num_groups,
                epsilon=epsilon,
            )
        )

        for layer_idx in range(1, num_layers):
            layers.append(
                TabNet(
                    feature_columns=None,
                    num_features=output_dim[layer_idx - 1],
                    feature_dim=feature_dim[layer_idx],
                    output_dim=output_dim[layer_idx],
                    num_decision_steps=num_decision_steps,
                    relaxation_factor=relaxation_factor,
                    sparsity_coefficient=sparsity_coefficient,
                    norm_type=norm_type,
                    batch_momentum=batch_momentum,
                    virtual_batch_size=virtual_batch_size,
                    num_groups=num_groups,
                    epsilon=epsilon,
                )
            )

        self.tabnet_layers = layers

    def call(self, inputs, training=None):
        x = self.tabnet_layers[0](inputs, training=training)

        for layer_idx in range(1, self.num_layers):
            x = self.tabnet_layers[layer_idx](x, training=training)

        return x

    @property
    def tabnets(self):
        return self.tabnet_layers

    @property
    def feature_selection_masks(self):
        return [tabnet.feature_selection_masks for tabnet in self.tabnet_layers]

    @property
    def aggregate_feature_selection_mask(self):
        return [tabnet.aggregate_feature_selection_mask for tabnet in self.tabnet_layers]

In [28]:
class StackedTabNetClassifier(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_classes,
        num_layers=1,
        feature_dim=64,
        output_dim=64,
        num_features=None,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=2,
        epsilon=1e-5,
        multi_label=False,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        Stacked variant of the TabNet model, which stacks multiple TabNets into a singular model.
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_classes: Number of classes.
            num_layers: Number of TabNets to stack together.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer. Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
                Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'batch' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(StackedTabNetClassifier, self).__init__(**kwargs)

        self.num_classes = num_classes

        self.stacked_tabnet = StackedTabNet(
            feature_columns=feature_columns,
            num_layers=num_layers,
            feature_dim=feature_dim,
            output_dim=output_dim,
            num_features=num_features,
            num_decision_steps=num_decision_steps,
            relaxation_factor=relaxation_factor,
            sparsity_coefficient=sparsity_coefficient,
            norm_type=norm_type,
            batch_momentum=batch_momentum,
            virtual_batch_size=virtual_batch_size,
            num_groups=num_groups,
            epsilon=epsilon,
        )
        if multi_label:

            self.clf = tf.keras.layers.Dense(num_classes, activation="sigmoid", use_bias=False)

        else:

            self.clf = tf.keras.layers.Dense(num_classes, activation="softmax", use_bias=False)

    def call(self, inputs, training=None):
        self.activations = self.stacked_tabnet(inputs, training=training)
        out = self.clf(self.activations)

        return out

In [29]:
class StackedTabNetRegressor(tf.keras.Model):
    def __init__(
        self,
        feature_columns,
        num_regressors,
        num_layers=1,
        feature_dim=64,
        output_dim=64,
        num_features=None,
        num_decision_steps=5,
        relaxation_factor=1.5,
        sparsity_coefficient=1e-5,
        norm_type="group",
        batch_momentum=0.98,
        virtual_batch_size=None,
        num_groups=2,
        epsilon=1e-5,
        **kwargs
    ):
        """
        Tensorflow 2.0 implementation of [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)
        Stacked variant of the TabNet model, which stacks multiple TabNets into a singular model.
        # Hyper Parameter Tuning (Excerpt from the paper)
        We consider datasets ranging from ∼10K to ∼10M training points, with varying degrees of fitting
        difficulty. TabNet obtains high performance for all with a few general principles on hyperparameter
        selection:
            - Most datasets yield the best results for Nsteps ∈ [3, 10]. Typically, larger datasets and
            more complex tasks require a larger Nsteps. A very high value of Nsteps may suffer from
            overfitting and yield poor generalization.
            - Adjustment of the values of Nd and Na is the most efficient way of obtaining a trade-off
            between performance and complexity. Nd = Na is a reasonable choice for most datasets. A
            very high value of Nd and Na may suffer from overfitting and yield poor generalization.
            - An optimal choice of γ can have a major role on the overall performance. Typically a larger
            Nsteps value favors for a larger γ.
            - A large batch size is beneficial for performance - if the memory constraints permit, as large
            as 1-10 % of the total training dataset size is suggested. The virtual batch size is typically
            much smaller than the batch size.
            - Initially large learning rate is important, which should be gradually decayed until convergence.
        Args:
            feature_columns: The Tensorflow feature columns for the dataset.
            num_regressors: Number of regressors.
            num_layers: Number of TabNets to stack together.
            feature_dim (N_a): Dimensionality of the hidden representation in feature
                transformation block. Each layer first maps the representation to a
                2*feature_dim-dimensional output and half of it is used to determine the
                nonlinearity of the GLU activation where the other half is used as an
                input to GLU, and eventually feature_dim-dimensional output is
                transferred to the next layer. Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            output_dim (N_d): Dimensionality of the outputs of each decision step, which is
                later mapped to the final classification or regression output.
                Can be either a single int, or a list of
                integers. If a list, must be of same length as the number of layers.
            num_features: The number of input features (i.e the number of columns for
                tabular data assuming each feature is represented with 1 dimension).
            num_decision_steps(N_steps): Number of sequential decision steps.
            relaxation_factor (gamma): Relaxation factor that promotes the reuse of each
                feature at different decision steps. When it is 1, a feature is enforced
                to be used only at one decision step and as it increases, more
                flexibility is provided to use a feature at multiple decision steps.
            sparsity_coefficient (lambda_sparse): Strength of the sparsity regularization.
                Sparsity may provide a favorable inductive bias for convergence to
                higher accuracy for some datasets where most of the input features are redundant.
            norm_type: Type of normalization to perform for the model. Can be either
                'batch' or 'group'. 'group' is the default.
            batch_momentum: Momentum in ghost batch normalization.
            virtual_batch_size: Virtual batch size in ghost batch normalization. The
                overall batch size should be an integer multiple of virtual_batch_size.
            num_groups: Number of groups used for group normalization.
            epsilon: A small number for numerical stability of the entropy calculations.
        """
        super(StackedTabNetRegressor, self).__init__(**kwargs)

        self.num_regressors = num_regressors

        self.stacked_tabnet = StackedTabNet(
            feature_columns=feature_columns,
            num_layers=num_layers,
            feature_dim=feature_dim,
            output_dim=output_dim,
            num_features=num_features,
            num_decision_steps=num_decision_steps,
            relaxation_factor=relaxation_factor,
            sparsity_coefficient=sparsity_coefficient,
            norm_type=norm_type,
            batch_momentum=batch_momentum,
            virtual_batch_size=virtual_batch_size,
            num_groups=num_groups,
            epsilon=epsilon,
        )

        self.regressor = tf.keras.layers.Dense(num_regressors, use_bias=False)

    def call(self, inputs, training=None):
        self.activations = self.tabnet(inputs, training=training)
        out = self.regressor(self.activations)
        return outl

## 学習

In [30]:
N_STARTS = 1
N_SPLITS = 10

class Objective:
    def __init__(self):
        self.best_submit_df = None
        self._submit_df = None

    def __call__(self, trial):
        use_existing_features = trial.suggest_int("use_existing_features", 0, 1)
        num_g_cols = trial.suggest_int("num_g_cols", 100, 771)
        num_c_cols = trial.suggest_int("num_c_cols", 10, 99)
        variance_threshold = trial.suggest_float("variance_threshold", 0.0, 1.0)

        train_df = pd.read_csv("../input/lish-moa/train_features.csv")
        test_df = pd.read_csv("../input/lish-moa/test_features.csv")
        target_df = pd.read_csv("../input/lish-moa/train_targets_scored.csv")
        submit_df = pd.read_csv("../input/lish-moa/sample_submission.csv")

        train = train_df.copy()
        test = test_df.copy()

        # encoding
        train.loc[:, "cp_dose"] = train.loc[:, "cp_dose"].map({"D1": 0, "D2": 1})
        test.loc[:, "cp_dose"] = test.loc[:, "cp_dose"].map({"D1": 0, "D2": 1})

        # ctl_vehicle 除外
        target_df = target_df.loc[train["cp_type"] != "ctl_vehicle"].reset_index(drop=True)
        train = train.loc[train["cp_type"] != "ctl_vehicle"].reset_index(drop=True)

        train = train.drop("cp_type", axis=1)
        test = test.drop("cp_type", axis=1)

        del train["sig_id"]
        del target_df["sig_id"]
        del test["sig_id"]

        g_cols = [col for col in train_df.columns if col.startswith("g-")]
        c_cols = [col for col in train_df.columns if col.startswith("c-")]

        # Rank Gauss
        for col in g_cols + c_cols:
            transformer = QuantileTransformer(n_quantiles=100, random_state=random_seed, output_distribution="normal")

            vec_len = len(train[col].values)
            vec_len_test = len(test[col].values)

            raw_vec = train[col].values.reshape(vec_len, 1)
            transformer.fit(raw_vec)

            train[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
            test[col] = transformer.transform(test[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

        # PCA
        # g-
        n_comp = num_g_cols

        data = pd.concat([pd.DataFrame(train[g_cols]), pd.DataFrame(test[g_cols])])
        data2 = PCA(n_components=n_comp, random_state=random_seed).fit_transform(data[g_cols])
        train2 = data2[: train.shape[0]]
        test2 = data2[-test.shape[0] :]

        train2 = pd.DataFrame(train2, columns=[f"pca_G-{i}" for i in range(n_comp)])
        test2 = pd.DataFrame(test2, columns=[f"pca_G-{i}" for i in range(n_comp)])

        if use_existing_features == 0:
            train.drop(g_cols, axis=1, inplace=True)
            test.drop(g_cols, axis=1, inplace=True)

        train = pd.concat((train, train2), axis=1)
        test = pd.concat((test, test2), axis=1)

        # c-
        n_comp = num_c_cols

        data = pd.concat([pd.DataFrame(train[c_cols]), pd.DataFrame(test[c_cols])])
        data2 = PCA(n_components=n_comp, random_state=random_seed).fit_transform(data[c_cols])
        train2 = data2[: train.shape[0]]
        test2 = data2[-test.shape[0] :]

        train2 = pd.DataFrame(train2, columns=[f"pca_C-{i}" for i in range(n_comp)])
        test2 = pd.DataFrame(test2, columns=[f"pca_C-{i}" for i in range(n_comp)])

        if use_existing_features == 0:
            train.drop(c_cols, axis=1, inplace=True)
            test.drop(c_cols, axis=1, inplace=True)

        train = pd.concat((train, train2), axis=1)
        test = pd.concat((test, test2), axis=1)

        # Variance Encoding
        var_thresh = VarianceThreshold(threshold=variance_threshold)

        data = train.append(test)
        data_transformed = var_thresh.fit_transform(data.iloc[:, 2:])

        train_transformed = data_transformed[: train.shape[0]]
        test_transformed = data_transformed[-test.shape[0] :]

        train = pd.DataFrame(train[["cp_time", "cp_dose"]].values.reshape(-1, 2), columns=["cp_time", "cp_dose"])
        train = pd.concat([train, pd.DataFrame(train_transformed)], axis=1)

        test = pd.DataFrame(test[["cp_time", "cp_dose"]].values.reshape(-1, 2), columns=["cp_time", "cp_dose"])
        test = pd.concat([test, pd.DataFrame(test_transformed)], axis=1)

        res = target_df.copy()
        submit_df.loc[:, target_df.columns] = 0
        res.loc[:, target_df.columns] = 0

        for seed in range(N_STARTS):
            for n, (tr, te) in enumerate(
                MultilabelStratifiedKFold(n_splits=N_SPLITS, random_state=seed, shuffle=True).split(
                    target_df, target_df
                )
            ):
                start_time = time()

                # model = create_model()
                model = StackedTabNetClassifier(
                    feature_columns=None,
                    num_classes=206,
                    num_layers=2,
                    feature_dim=128,
                    output_dim=64,
                    num_features=len(train.columns),
                    num_decision_steps=1,
                    relaxation_factor=1.5,
                    sparsity_coefficient=1e-5,
                    batch_momentum=0.98,
                    virtual_batch_size=None,
                    norm_type="group",
                    num_groups=-1,
                    multi_label=True,
                )

                model.compile(
                    optimizer=tfa.optimizers.Lookahead(tf.optimizers.Adam(1e-3), sync_period=10),
                    loss="binary_crossentropy",
                )

                reduce_lr_loss = ReduceLROnPlateau(
                    monitor="val_loss", factor=0.1, patience=3, verbose=0, min_delta=1e-4, mode="min"
                )
                early_stopping = EarlyStopping(
                    monitor="val_loss",
                    patience=10,
                    mode="min",
                    verbose=0,
                    min_delta=1e-4,
                    restore_best_weights=True,
                )
                model.fit(
                    train.values[tr],
                    target_df.values[tr],
                    validation_data=(train.values[te], target_df.values[te]),
                    epochs=100,
                    batch_size=128,
                    callbacks=[reduce_lr_loss, early_stopping],
                    verbose=0,
                )

                test_predict = model.predict(test.values)
                val_predict = model.predict(train.values[te])

                submit_df.loc[:, target_df.columns] += test_predict / (N_SPLITS * N_STARTS)
                res.loc[te, target_df.columns] += val_predict / N_STARTS

                fold_score = metric(target_df.loc[te].values, val_predict)

                print(
                    f"[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] TabNet: Seed {seed}, Fold {n}:",
                    fold_score,
                )

                K.clear_session()
                del model
                x = gc.collect()

        self._submit_df = submit_df
        score = metric(target_df.values, res.values)

        return score

    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_submit_df = self._submit_df.copy()

In [31]:
objective = Objective()
study = optuna.create_study()
study.optimize(objective, n_trials=20, callbacks=[objective.callback])

/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error



[01:02] TabNet: Seed 0, Fold 0: 0.01757906165393488
[01:01] TabNet: Seed 0, Fold 1: 0.01752760893410638
[01:00] TabNet: Seed 0, Fold 2: 0.018071846274350396
[01:00] TabNet: Seed 0, Fold 3: 0.017659019682542392
[01:00] TabNet: Seed 0, Fold 4: 0.017646695727423913
[01:00] TabNet: Seed 0, Fold 5: 0.017408326743300975
[01:00] TabNet: Seed 0, Fold 6: 0.017395727243024902
[00:55] TabNet: Seed 0, Fold 7: 0.017685169707646324
[01:00] TabNet: Seed 0, Fold 8: 0.017516096167437677
[00:57] TabNet: Seed 0, Fold 9: 0.017703998435106994


[I 2020-10-14 01:54:56,059] Trial 0 finished with value: 0.017619374860349438 and parameters: {'use_existing_features': 1, 'num_g_cols': 766, 'num_c_cols': 79, 'variance_threshold': 0.976660465819299}. Best is trial 0 with value: 0.017619374860349438.
/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error



[01:05] TabNet: Seed 0, Fold 0: 0.017420513414620586
[00:59] TabNet: Seed 0, Fold 1: 0.017488383723993146
[00:58] TabNet: Seed 0, Fold 2: 0.01757770638346513
[01:01] TabNet: Seed 0, Fold 3: 0.01754060148701097
[01:05] TabNet: Seed 0, Fold 4: 0.01761600294238051
[01:05] TabNet: Seed 0, Fold 5: 0.01725024472909532
[01:05] TabNet: Seed 0, Fold 6: 0.017366048287218782
[01:04] TabNet: Seed 0, Fold 7: 0.017393433465548547
[01:04] TabNet: Seed 0, Fold 8: 0.01740640025010772
[01:00] TabNet: Seed 0, Fold 9: 0.01771406547206126


[I 2020-10-14 02:05:44,903] Trial 1 finished with value: 0.0174773554332808 and parameters: {'use_existing_features': 0, 'num_g_cols': 312, 'num_c_cols': 96, 'variance_threshold': 0.8974948405168504}. Best is trial 1 with value: 0.0174773554332808.
/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error



...

[I 2020-10-14 04:55:27,281] Trial 17 finished with value: 0.017480911479618826 and parameters: {'use_existing_features': 0, 'num_g_cols': 245, 'num_c_cols': 25, 'variance_threshold': 0.8860892069633682}. Best is trial 12 with value: 0.017327161803306276.
/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error



[01:00] TabNet: Seed 0, Fold 0: 0.01742475430500862
[00:59] TabNet: Seed 0, Fold 1: 0.01748089095964688
[01:01] TabNet: Seed 0, Fold 2: 0.01744797102412869
[01:05] TabNet: Seed 0, Fold 3: 0.01713495066743928
[01:06] TabNet: Seed 0, Fold 4: 0.017773837048227333
[01:00] TabNet: Seed 0, Fold 5: 0.017177846907382128
[00:59] TabNet: Seed 0, Fold 6: 0.017630273226786685
[00:59] TabNet: Seed 0, Fold 7: 0.01751233727073034
[01:05] TabNet: Seed 0, Fold 8: 0.017536535779503693
[01:01] TabNet: Seed 0, Fold 9: 0.017598744512481425


[I 2020-10-14 05:06:04,135] Trial 18 finished with value: 0.01747182035171014 and parameters: {'use_existing_features': 0, 'num_g_cols': 179, 'num_c_cols': 63, 'variance_threshold': 0.39913465404686654}. Best is trial 12 with value: 0.017327161803306276.
/home/ubuntu/work/tensorflow-gpu/.venv/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning:

Pass shuffle=True, random_state=0 as keyword args. From version 0.25 passing these as positional arguments will result in an error



[00:57] TabNet: Seed 0, Fold 0: 0.017643595666627095
[00:56] TabNet: Seed 0, Fold 1: 0.017839851796784736
[00:56] TabNet: Seed 0, Fold 2: 0.01812992860749519
[00:56] TabNet: Seed 0, Fold 3: 0.017764190773669006
[00:56] TabNet: Seed 0, Fold 4: 0.01768032295081006
[00:56] TabNet: Seed 0, Fold 5: 0.017611706774815217
[00:56] TabNet: Seed 0, Fold 6: 0.017702939139905262
[00:56] TabNet: Seed 0, Fold 7: 0.017760825945543512
[00:56] TabNet: Seed 0, Fold 8: 0.017819746141297707
[00:53] TabNet: Seed 0, Fold 9: 0.01799897227465094


[I 2020-10-14 05:15:45,681] Trial 19 finished with value: 0.017795220555865028 and parameters: {'use_existing_features': 1, 'num_g_cols': 288, 'num_c_cols': 83, 'variance_threshold': 0.643198084810188}. Best is trial 12 with value: 0.017327161803306276.


## Result

In [32]:
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value: 0.017327161803306276
  Params: 
    use_existing_features: 0
    num_g_cols: 103
    num_c_cols: 99
    variance_threshold: 0.9996082435895635


In [33]:
optuna.importance.get_param_importances(study)

OrderedDict([('num_g_cols', 0.659198475777005),
             ('variance_threshold', 0.16505603484007159),
             ('use_existing_features', 0.14021386189225393),
             ('num_c_cols', 0.03553162749066957)])

## 補正

## Output